# step 0

conda environment:
    
conda create -n week6 -c bioconda -c anaconda fastqc bismark samtools bowtie2 igv

data:
 
SRA: https://www.ncbi.nlm.nih.gov/sra/

paper: https://www-nature-com.proxy1.library.jhu.edu/articles/s41588-017-0003-x#Sec14

two datasets, downloaded to week6-files folder: E4.0ICM rep1 and STEM-seq E5.5Epi rep1

You will need to determine which file belongs to which sample using GEO (look for the SRA link on each sample’s webpage).

SRR3083926 is ------- GSM2027229: STEM-seq E4.0ICM rep1; Mus musculus; Bisulfite-Seq (SO THIS IS ICM)

SRR3083929 is ------- GSM2027231: STEM-seq E5.5Epi rep1; Mus musculus; Bisulfite-Seq (SO THIS IS EPIBLAST)


some command line things:
    
    less -S SRR3083926_1.chr6.fastq
    fastqc SRR3083929_1.chr6.fastq
    fastqc SRR3083929_2.chr6.fastq
    fastqc SRR3083926_1.chr6.fastq
    fastqc SRR3083926_2.chr6.fastq
    
<strike>honestly I dont really notice anything I didnt expect? The read quality drastically declines at the end but isnt that what we would expect?

oops jk the 1 strands have no Cs, and the other strand has no Gs. This totally makes sense in the context of bisulfite seq, since the treatment converts unmethylated cytosines to uracil, and the complementary strand would have the inverse effect.

# step 1: reference genome

make a reference genome index:
    bismark_genome_preparation chr6.fa
    
didnt work, need to make a folder
    mkdir chr6
    mv chr6.fa chr6/chr6.fa
    
bismark_genome_preparation chr6

ok this one worked


# step 2: mapping

mapping my two experiments to the genome using bismark

bismark --parallel 4 chr6 -1 methylation_fastq/SRR3083926_1.chr6.fastq -2 methylation_fastq/SRR3083926_2.chr6.fastq

bismark --parallel 4 chr6 -1 methylation_fastq/SRR3083929_1.chr6.fastq -2 methylation_fastq/SRR3083929_2.chr6.fastq

# step 3: sorting and indexing

before I can load it into the IGV I need a sorted bam and an index:

for sample in SRR3083926_1 SRR3083929_1
do
    samtools sort ${sample}.chr6_bismark_bt2_pe.bam -o ${sample}.sorted.bam
    samtools index ${sample}.sorted.bam
done

# step 4: extracting methylation data

In [ ]:
now I need to extract the methylation data: 
    bismark_methylation_extractor --bedgraph --comprehensive SRR3083929_1.sorted.bam
    bismark_methylation_extractor --bedgraph --comprehensive ivg_26/SRR3083926_1.sorted.bam

In [23]:
methyl_dict = {}
count = 0

E4 = open('/Users/cmdb/qbb2020-answers/week6/week6_files/SRR3083926_1.sorted.bedGraph')

for line in E4:
    count += 1
    if line.startswith('track'):
        continue
    fields = line.strip('\r\n').split('\t')
    key = fields[1]
    methyl = fields[3]
    if methyl == '0':
        continue
    else: 
        #methyl_dict.update(key : methyl)
        methyl_dict[key] = [methyl]
count1 = 0
for key, value in methyl_dict.items():
    count1 += 1
    print(key, value)
    if count1 > 10:
        break

50006221 ['100']
50008106 ['100']
50011269 ['80']
50012922 ['50']
50014554 ['100']
50014758 ['50']
50014774 ['50']
50015088 ['100']
50016957 ['20']
50018339 ['33.3333333333333']
50018812 ['66.6666666666667']


In [24]:
E55 = open('/Users/cmdb/qbb2020-answers/week6/week6_files/SRR3083929_1.sorted.bedGraph')

for line in E55:
    count += 1
    if line.startswith('track'):
        continue
    fields = line.strip('\r\n').split('\t')
    key1 = fields[1]
    methyl1 = fields[3]
    if key1 in methyl_dict:
        methyl_dict[key1].append(methyl1)
    else:
        continue
count1 = 0
for key, value in methyl_dict.items():
    count1 += 1
    print(key, value)
    if count1 > 10:
        break

50006221 ['100']
50008106 ['100', '100']
50011269 ['80']
50012922 ['50']
50014554 ['100']
50014758 ['50']
50014774 ['50']
50015088 ['100']
50016957 ['20']
50018339 ['33.3333333333333', '100']
50018812 ['66.6666666666667']


In [28]:
fold_dict = {}
for key, value in methyl_dict.items():
    if len(value) == 2:
        fold = float(value[1])/float(value[0])
        fold_dict[key] = fold
    else:
        continue
count1 = 0
for key, value in fold_dict.items():
    count1 += 1
    print(key, value)
    if count1 > 10:
        break
    

50008106 1.0
50018339 3.000000000000003
50018960 1.0
50019058 0.222222222222222
50024598 1.0
50024610 1.0
50025197 0.5
50025242 1.8000000000000018
50035357 0.0
50036288 0.5
50036300 0.666666666666667


In [ ]:
13th column = gene name (key)
5th and 6th column = start and end


In [29]:
name_dict = {}

mm10 = open('/Users/cmdb/qbb2020-answers/week6/week6_files/mm10_refseq_genes_chr6_50M_60M.bed')

for line in mm10:
    fields = line.strip('\r\n').split('\t')
    key = fields[12]
    start = fields[4]
    end = fields[5]
    name_dict[key] = [start, end]
    
count1 = 0
for key, value in name_dict.items():
    count1 += 1
    print(key, value)
    if count1 > 10:
        break

Mpp6 ['50110240', '50198598']
Dfna5 ['50207402', '50261769']
Osbpl3 ['50293326', '50456170']
Cycs ['50562562', '50566474']
5430402O13Rik ['50566642', '50594865']
Mir6371 ['50572397', '50572508']
4921507P07Rik ['50573303', '50596590']
Npvf ['50650870', '50654393']
C530044C16Rik ['50776114', '50814894']
Mir148a ['51269811', '51269910']
Gm6559 ['51379709', '51392791']


In [32]:
for key, value in name_dict.items():
    for key1, value1 in fold_dict.items():
        if key1 >= value[0] and key1 <= value[1]:
            name_dict[key].append(value1)

count1 = 0
for key, value in name_dict.items():
    count1 += 1
    print(key, value)
    if count1 > 10:
        break
            
    

Mpp6 ['50110240', '50198598', 2.5, 2.0, 1.0, 1.0, 1.0, 0.0, 2.4000000000000026, 0.2, 1.0, 0.666666666666667, 1.0, 2.0, 0.583333333333333, 5.0, 2.0, 1.0, 3.000000000000003, 1.0, 1.0, 1.0, 1.2307692307692322, 6.000000000000008, 20.52000000000002, 1.5000000000000004, 0.0, 2.9166666666666674, 5.0, 0.0, 0.0, 1.4999999999999993, 1.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 2.0, 1.25, 0.0, 1.0, 1.0, 1.4999999999999993, 2.5, 2.5, 3.000000000000003, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.666666666666666, 3.4999999999999964, 0.666666666666666, 1.0, 1.1249999999999998, 1.1249999999999998, 3.4999999999999964, 3.4999999999999964, 9.000000000000009, 0.5, 0.5, 1.0, 1.0, 5.0, 2.0, 0.428571428571429, 1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.333333333333333, 1.0, 9.000000000000009, 0.25, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.5, 4.0, 3.000000000000003, 2.3333333333333313]
Dfna5 ['50207402', '50261769', 1.0, 1.0, 1.0, 4.0, 3.000000000000003, 4.0, 0.0, 1.4999999999999993, 0.0, 5.0, 1.0, 1.0, 1.0, 2.400000000

In [34]:
name_fold_dict = {}
for key, value in name_dict.items():
    if len(value)==3:
        name_fold_dict[key]=value[2]

for key, value in name_fold_dict.items():
    print(key, value)

        

Npvf 1.0
Mir148a 0.0
Halr1 1.0
Hoxa2 0.0
Hoxaas2 0.0
Hoxa5 1.0
Mir196b 0.0
Hoxa11 0.0
Hoxa13 0.0
Hottip 0.0
Ghrhr 2.0
Vmn1r4 0.0
Vmn1r5 0.5
Vmn1r11 0.8
Vmn1r13 0.333333333333333
Vmn1r15 0.0
Vmn1r27 1.0
Vmn1r29 1.0
Vmn1r30 1.0
